# Sudeste Module 3 Exam

### Instructions
Modify this notebook to compose your model. You may use additional `.py` files within this folder, `src/` for composing your framework.

It is discouraged to rename the `notebook.ipynb` for this particular task.

<div style="background-color: #f7f7f7; border: 1px solid #e1e1e1; border-radius: 5px; padding: 15px; max-width: 400px;">
    <h2 style="color: #333333; font-family: Arial, sans-serif;">Submitted by Group 6</h2>
    <ul style="color: #555555; font-family: Arial, sans-serif; font-size: 16px;">
        <li>Aguaviva, Christopher</li>
        <li>Baluyot, Kent Ryan</li>
        <li>Berida, Ronabelle</li>
    </ul>
</div>


### <span style="color:Orange;">  Table of Contents: </span>

I.    [Executive Summary](#summary)

II.   [Problem Statement](#problem)

III.  [Objective](#objective)

IV.   [Data Cleaning](#datacleaning)

V.    [Visualizations and EDA](#eda)

VI.   [ARIMA Model](#arima)

VII.  [Deep Learning Model](#dl)
  
VIII. [Comparison](#comparison)

IX.   [Conclusion and Recommendations](#conclusion)

***

## <span style="color:Orange;"> I. Executive Summary </span>  <a name="summary"></a>
`edit contents`

## <span style="color:Orange;"> II. Problem Statement </span>  <a name="problem"></a>  
`edit contents`

## <span style="color:Orange;"> III. Objective </span>  <a name="objective"></a>  
`edit contents`

## <span style="color:Orange;"> IV. Data Cleaning </span>  <a name="datacleaning"></a>  

Importing Modules

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import tensorflow as tf

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
#from datetime import datetime

pd.set_option('display.max_columns', 40)   # displays max columns of the DataFrame 

## <span style="color:Orange;"> V. Visualizations and EDA </span>  <a name="eda"></a>  

## <span style="color:Orange;"> VI. ARIMA Model </span>  <a name="arima"></a>  
`edit contents`

## <span style="color:Orange;"> VII. Deep Learning Model </span>  <a name="dl"></a>  
`edit contents`

## <span style="color:Orange;"> VIII. Comparison </span>  <a name="comparison"></a>  
<i> Comparison between the results of the two specified models: ARIMA and Deep Learning Models, and assessing which model performs better in <b> Predicting and Classification </b> </i>  



## <span style="color:Orange;"> IX. Conclusion and Recommendation </span> <a name="conclusion"></a>  
`edit contents`